<a href="https://colab.research.google.com/github/adindaops/Machine-Learning/blob/main/Week7_Distributed%20XGBoost%20with%20Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ray

     |████████████████████████████████| 54.7 MB 32 kB/s 
     |████████████████████████████████| 72 kB 485 kB/s 


In [2]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 5.1 MB/s 


In [3]:
pip install sklearn

In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 08:34:27,131	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-12 08:34:29,278	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=287) [08:34:29] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=287) [08:34:29] Tree method is automatically selected to be 'approx' for distributed training.
(pid=311) [08:34:29] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 08:34:30,964	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-12 08:34:33,427	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-12 08:34:43,309	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-12 08:34:43,338	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-12 08:34:43,340	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_627ec_00000,PENDING,,0.0202656,6,0.668731
train_model_627ec_00001,PENDING,,0.000122029,3,0.856801
train_model_627ec_00002,PENDING,,0.000915446,5,0.669293
train_model_627ec_00003,PENDING,,0.00103306,2,0.640502


(pid=391) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=391)   warnings.warn(LEGACY_WARNING)
(pid=391) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=391)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=391) 2021-11-12 08:34:45,747	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=391) 2021-11-12 08:34:48,473	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Trial name,status,loc,eta,max_depth,subsample
train_model_627ec_00000,RUNNING,172.28.0.2:391,0.0202656,6,0.668731
train_model_627ec_00001,PENDING,,0.000122029,3,0.856801
train_model_627ec_00002,PENDING,,0.000915446,5,0.669293
train_model_627ec_00003,PENDING,,0.00103306,2,0.640502


Result for train_model_627ec_00000:
  date: 2021-11-12_08-34-49
  done: false
  experiment_id: 05bb1c58a33341749ef5e0244d0b4b53
  hostname: 8b6bdd6e6c30
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 391
  time_since_restore: 3.8321926593780518
  time_this_iter_s: 3.8321926593780518
  time_total_s: 3.8321926593780518
  timestamp: 1636706089
  timesteps_since_restore: 0
  train-error: 0.043937
  train-logloss: 0.676039
  training_iteration: 1
  trial_id: 627ec_00000
  
Result for train_model_627ec_00000:
  date: 2021-11-12_08-34-49
  done: true
  experiment_id: 05bb1c58a33341749ef5e0244d0b4b53
  experiment_tag: 0_eta=0.020266,max_depth=6,subsample=0.66873
  hostname: 8b6bdd6e6c30
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 391
  time_since_restore: 3.914759397506714
  time_this_iter_s: 0.0060422420501708984
  time_total_s: 3.914759397506714
  timestamp: 1636706089
  timesteps_since_restore: 0
  train-error: 0.019332
  train-logloss: 0.546542
  training_iterat

...

Result for train_model_627ec_00003:
  date: 2021-11-12_08-35-07
  done: false
  experiment_id: 1dbaa6bdae934464bf7b3d6a44a8691c
  hostname: 8b6bdd6e6c30
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 707
  time_since_restore: 3.8301455974578857
  time_this_iter_s: 3.8301455974578857
  time_total_s: 3.8301455974578857
  timestamp: 1636706107
  timesteps_since_restore: 0
  train-error: 0.056239
  train-logloss: 0.69232
  training_iteration: 1
  trial_id: 627ec_00003
  
Result for train_model_627ec_00003:
  date: 2021-11-12_08-35-07
  done: true
  experiment_id: 1dbaa6bdae934464bf7b3d6a44a8691c
  experiment_tag: 3_eta=0.0010331,max_depth=2,subsample=0.6405
  hostname: 8b6bdd6e6c30
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 707
  time_since_restore: 3.9002697467803955
  time_this_iter_s: 0.006918191909790039
  time_total_s: 3.9002697467803955
  timestamp: 1636706107
  timesteps_since_restore: 0
  train-error: 0.040422
  train-logloss: 0.684884
  training_iterat

(pid=707) 2021-11-12 08:35:07,418	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.92 seconds (1.12 pure XGBoost training time).


Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_627ec_00000,TERMINATED,172.28.0.2:391,0.0202656,6,0.668731,10,3.91476,0.546542,0.019332
train_model_627ec_00001,TERMINATED,172.28.0.2:497,0.000122029,3,0.856801,10,3.90975,0.692107,0.022847
train_model_627ec_00002,TERMINATED,172.28.0.2:601,0.000915446,5,0.669293,10,3.89561,0.685395,0.017575
train_model_627ec_00003,TERMINATED,172.28.0.2:707,0.00103306,2,0.640502,10,3.90027,0.684884,0.040422


2021-11-12 08:35:07,583	INFO tune.py:630 -- Total run time: 24.28 seconds (24.07 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0009154463715588816, 'subsample': 0.6692930498525176, 'max_depth': 5}
